# Pycaret을 이용한 베이스라인 모델

# [0] 모듈 로드

- 서드파트 모듈 설치

In [ ]:
!pip install -q pycaret
!pip install --upgrade -q xgboost
!pip install -q catboost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 484.7/484.7 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.9/81.9 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.9/79.9 MB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.8/11.8 MB 87.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.4/73.4 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 72.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 160.5/160.5 kB 19.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.8/106.8 kB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.4/34.4 MB 21.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 60.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 70.3 MB/s et

- 라이브러리 로드

In [ ]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler

!pip install --upgrade -q xgboost
!pip install -q catboost
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_breast_cancer
from sklearn.metrics import accuracy_score, mean_squared_error
from sklearn.ensemble import RandomForestClassifier

import warnings

warnings.filterwarnings('ignore')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# [1] 데이터 로드

In [ ]:
train = pd.read_csv('/content/drive/MyDrive/Dacon/23_대구교통사고피해예측AI경진대회/data/train.csv', encoding = 'utf-8')
test = pd.read_csv('/content/drive/MyDrive/Dacon/23_대구교통사고피해예측AI경진대회/data/test.csv', encoding = 'utf-8')
sub = pd.read_csv('/content/drive/MyDrive/Dacon/23_대구교통사고피해예측AI경진대회/data/sample_submission.csv', encoding = 'utf-8')

In [ ]:
train.columns

Index(['ID', '사고일시', '요일', '기상상태', '시군구', '도로형태', '노면상태', '사고유형',
       '사고유형 - 세부분류', '법규위반', '가해운전자 차종', '가해운전자 성별', '가해운전자 연령', '가해운전자 상해정도',
       '피해운전자 차종', '피해운전자 성별', '피해운전자 연령', '피해운전자 상해정도', '사망자수', '중상자수',
       '경상자수', '부상자수', 'ECLO'],
      dtype='object')

In [ ]:
train.head()
# 피해

,ID,사고일시,요일,기상상태,시군구,도로형태,노면상태,사고유형,사고유형 - 세부분류,법규위반,...,가해운전자 상해정도,피해운전자 차종,피해운전자 성별,피해운전자 연령,피해운전자 상해정도,사망자수,중상자수,경상자수,부상자수,ECLO
0,ACCIDENT_00000,2019-01-01 00,화요일,맑음,대구광역시 중구 대신동,단일로 - 기타,건조,차대사람,길가장자리구역통행중,안전운전불이행,...,상해없음,보행자,여,70세,중상,0,1,0,0,5
1,ACCIDENT_00001,2019-01-01 00,화요일,흐림,대구광역시 달서구 감삼동,단일로 - 기타,건조,차대사람,보도통행중,기타,...,상해없음,보행자,남,61세,경상,0,0,1,0,3
2,ACCIDENT_00002,2019-01-01 01,화요일,맑음,대구광역시 수성구 두산동,단일로 - 기타,건조,차대사람,차도통행중,안전운전불이행,...,상해없음,보행자,남,38세,경상,0,0,1,0,3
3,ACCIDENT_00003,2019-01-01 02,화요일,맑음,대구광역시 북구 복현동,단일로 - 기타,건조,차대차,추돌,안전운전불이행,...,상해없음,승용,남,36세,중상,0,1,0,0,5
4,ACCIDENT_00004,2019-01-01 04,화요일,맑음,대구광역시 동구 신암동,단일로 - 기타,건조,차대차,추돌,안전운전불이행,...,상해없음,승용,남,52세,경상,0,0,1,0,3


In [ ]:
test.head()

,ID,사고일시,요일,기상상태,시군구,도로형태,노면상태,사고유형
0,ACCIDENT_39609,2022-01-01 01,토요일,맑음,대구광역시 수성구 상동,교차로 - 교차로안,건조,차대사람
1,ACCIDENT_39610,2022-01-01 01,토요일,맑음,대구광역시 수성구 지산동,단일로 - 기타,건조,차대사람
2,ACCIDENT_39611,2022-01-01 04,토요일,맑음,대구광역시 수성구 수성동2가,교차로 - 교차로안,건조,차대차
3,ACCIDENT_39612,2022-01-01 04,토요일,맑음,대구광역시 수성구 신매동,단일로 - 기타,건조,차대차
4,ACCIDENT_39613,2022-01-01 06,토요일,맑음,대구광역시 달서구 감삼동,교차로 - 교차로안,건조,차대차


# [2] 데이터 전처리

In [ ]:
list(test.columns)

['ID', '사고일시', '요일', '기상상태', '시군구', '도로형태', '노면상태', '사고유형']

In [ ]:
data = train[list(test.columns)]
data_y = train['ECLO']

## 2-1 피쳐 세분화

- 시군구 세분화

In [ ]:
split_sgg = data['시군구'].str.split(' ', expand=True)
data['구'] = split_sgg[1]
data['동'] = split_sgg[2]
data.drop(['시군구'], inplace = True, axis = 1)

In [ ]:
split_sgg = test['시군구'].str.split(' ', expand=True)
test['구'] = split_sgg[1]
test['동'] = split_sgg[2]
test.drop(['시군구'], inplace = True, axis = 1)

In [ ]:
data.head(2)

,ID,사고일시,요일,기상상태,도로형태,노면상태,사고유형,구,동
0,ACCIDENT_00000,2019-01-01 00,화요일,맑음,단일로 - 기타,건조,차대사람,중구,대신동
1,ACCIDENT_00001,2019-01-01 00,화요일,흐림,단일로 - 기타,건조,차대사람,달서구,감삼동


- 사고일시세분화

In [ ]:
split_time = data['사고일시'].str.split('-', expand=True)
tmp_time = split_time[2].str.split(' ', expand = True)
data['연'] = split_time[0].astype(int)
data['월'] = split_time[1].astype(int)
data['일'] = tmp_time[0].astype(int)
data['시간'] = tmp_time[1].astype(int)

data.drop(['사고일시'], inplace = True, axis = 1)

In [ ]:
split_time = test['사고일시'].str.split('-', expand=True)
tmp_time = split_time[2].str.split(' ', expand = True)
test['연'] = split_time[0].astype(int)
test['월'] = split_time[1].astype(int)
test['일'] = tmp_time[0].astype(int)
test['시간'] = tmp_time[1].astype(int)

test.drop(['사고일시'], inplace = True, axis = 1)

In [ ]:
data.head(2)

,ID,요일,기상상태,도로형태,노면상태,사고유형,구,동,연,월,일,시간
0,ACCIDENT_00000,화요일,맑음,단일로 - 기타,건조,차대사람,중구,대신동,2019,1,1,0
1,ACCIDENT_00001,화요일,흐림,단일로 - 기타,건조,차대사람,달서구,감삼동,2019,1,1,0


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39609 entries, 0 to 39608
Data columns (total 12 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   ID      39609 non-null  object
 1   요일      39609 non-null  object
 2   기상상태    39609 non-null  object
 3   도로형태    39609 non-null  object
 4   노면상태    39609 non-null  object
 5   사고유형    39609 non-null  object
 6   구       39609 non-null  object
 7   동       39609 non-null  object
 8   연       39609 non-null  int64 
 9   월       39609 non-null  int64 
 10  일       39609 non-null  int64 
 11  시간      39609 non-null  int64 
dtypes: int64(4), object(8)
memory usage: 3.6+ MB


## 2-2. 인코딩

## 2-1. 라벨 인코딩

In [ ]:
qual_col = ['요일', '기상상태', '도로형태', '노면상태', '사고유형', '구', '동']

for i in qual_col:
    le = LabelEncoder()
    data[i]=le.fit_transform(data[i])
    test[i]=le.transform(test[i])

print('Done.')

Done.


In [ ]:
data.drop(['ID'], axis = 1, inplace = True)
test.drop(['ID'], axis = 1, inplace = True)

In [ ]:
data.columns == test.columns

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True])

# [3] 피쳐엔지니어링

In [ ]:
# 추후 보강

# [4] 모델링
- idea

In [ ]:
!pip install scipy

In [ ]:
from pycaret.regression import *

In [ ]:
X = pd.concat([data, data_y], axis = 1)
X.head(2)

,요일,기상상태,도로형태,노면상태,사고유형,구,동,연,월,일,시간,ECLO
0,6,2,6,0,0,7,40,2019,1,1,0,5
1,6,5,6,0,0,1,4,2019,1,1,0,3


In [ ]:
reg = setup(data = X # 데이터
      ,target = 'ECLO' # Target
      ,verbose = True # 진행 로그 출력
      ,train_size = 0.7 # 내부적으로 검증시 훈련데이터의 비율
      ,data_split_shuffle = True # 데이터 split시 데이터를 섞을것인지 여부
      ,normalize = True # 정규화 여부
      ,session_id = 22 # 난수 시드
      )

,Description,Value
0,Session id,22
1,Target,ECLO
2,Target type,Regression
3,Original data shape,"(39609, 12)"
4,Transformed data shape,"(39609, 12)"
5,Transformed train set shape,"(27726, 12)"
6,Transformed test set shape,"(11883, 12)"
7,Numeric features,11
8,Preprocess,True
9,Imputation type,simple


In [ ]:
models()

,Name,Reference,Turbo
ID,,,
lr,Linear Regression,sklearn.linear_model._base.LinearRegression,True
lasso,Lasso Regression,sklearn.linear_model._coordinate_descent.Lasso,True
ridge,Ridge Regression,sklearn.linear_model._ridge.Ridge,True
en,Elastic Net,sklearn.linear_model._coordinate_descent.Elast...,True
lar,Least Angle Regression,sklearn.linear_model._least_angle.Lars,True
llar,Lasso Least Angle Regression,sklearn.linear_model._least_angle.LassoLars,True
omp,Orthogonal Matching Pursuit,sklearn.linear_model._omp.OrthogonalMatchingPu...,True
br,Bayesian Ridge,sklearn.linear_model._bayes.BayesianRidge,True
ard,Automatic Relevance Determination,sklearn.linear_model._bayes.ARDRegression,False


In [ ]:
top_5_model = compare_models(fold = 5, round = 3, n_select = 5, sort = 'mse')

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
gbr,Gradient Boosting Regressor,2.128,9.958,3.151,0.029,0.458,0.621,1.834
lightgbm,Light Gradient Boosting Machine,2.144,10.055,3.166,0.019,0.460,0.623,2.264
br,Bayesian Ridge,2.147,10.120,3.177,0.013,0.462,0.630,0.056
lr,Linear Regression,2.147,10.121,3.177,0.013,0.462,0.630,2.036
ridge,Ridge Regression,2.147,10.121,3.177,0.013,0.462,0.630,0.096
lar,Least Angle Regression,2.147,10.121,3.177,0.013,0.462,0.630,0.054
omp,Orthogonal Matching Pursuit,2.152,10.151,3.182,0.010,0.463,0.632,0.048
catboost,CatBoost Regressor,2.160,10.188,3.187,0.006,0.464,0.626,6.026
lasso,Lasso Regression,2.154,10.255,3.198,-0.000,0.465,0.631,0.108
en,Elastic Net,2.154,10.255,3.198,-0.000,0.465,0.631,0.076


Processing:   0%|          | 0/89 [00:00<?, ?it/s]

In [ ]:
blended = blend_models(estimator_list = top_5_model,
             fold = 10
             ,optimize = 'rmsle'
            )

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,2.1401,9.0953,3.0158,0.0300,0.4607,0.6340
1,2.1123,8.8283,2.9712,0.0255,0.4601,0.6392
2,2.1813,12.1047,3.4792,0.0141,0.4620,0.6080
3,2.1761,11.8701,3.4453,0.0239,0.4638,0.6119
4,2.1231,9.8487,3.1383,0.0271,0.4551,0.6153
5,2.1259,9.4047,3.0667,0.0278,0.4572,0.6232
6,2.1404,9.3833,3.0632,0.0214,0.4594,0.6312
7,2.0812,8.8244,2.9706,0.0282,0.4522,0.6196
8,2.1391,10.5368,3.2460,0.0210,0.4642,0.6420


Processing:   0%|          | 0/6 [00:00<?, ?it/s]

In [ ]:
model = finalize_model(blended)
model

Pipeline(memory=Memory(location=None),
         steps=[('numerical_imputer',
                 TransformerWrapper(include=['요일', '기상상태', '도로형태', '노면상태',
                                             '사고유형', '구', '동', '연', '월', '일',
                                             '시간'],
                                    transformer=SimpleImputer())),
                ('categorical_imputer',
                 TransformerWrapper(include=[],
                                    transformer=SimpleImputer(strategy='most_frequent'))),
                ('normalize', TransformerWrapper(transformer=StandardScaler()...
                 TransformerWrapper(transformer=CleanColumnNames())),
                ('actual_estimator',
                 VotingRegressor(estimators=[('Gradient Boosting Regressor',
                                              GradientBoostingRegressor(random_state=22)),
                                             ('Light Gradient Boosting Machine',
                                              LGBMRegressor(n_jobs=-1,
                                                            random_state=22)),
                                             ('Bayesian Ridge',
                                              BayesianRidge()),
                                             ('Linear Regression',
                                              LinearRegression(n_jobs=-1)),
                                             ('Ridge Regression',
                                              Ridge(random_state=22))],
                                 n_jobs=-1))])

In [ ]:
y_pred = predict_model(model, data = test)
y_pred.head(5)

,요일,기상상태,도로형태,노면상태,사고유형,구,동,연,월,일,시간,prediction_label
0,5,2,1,0,0,6,99,2022,1,1,1,4.269143
1,5,2,6,0,0,6,168,2022,1,1,1,3.960298
2,5,2,1,0,1,6,117,2022,1,1,4,5.337770
3,5,2,6,0,1,6,126,2022,1,1,4,4.944801
4,5,2,1,0,1,1,4,2022,1,1,6,5.018953


In [ ]:
sub['ECLO'] = y_pred.prediction_label

In [ ]:
sub

,ID,ECLO
0,ACCIDENT_39609,4.269143
1,ACCIDENT_39610,3.960298
2,ACCIDENT_39611,5.337770
3,ACCIDENT_39612,4.944801
4,ACCIDENT_39613,5.018953
...,...,...
10958,ACCIDENT_50567,4.835231
10959,ACCIDENT_50568,4.515128
10960,ACCIDENT_50569,4.640059
10961,ACCIDENT_50570,4.742515


In [ ]:
sub.to_csv('/content/drive/MyDrive/Dacon/23_대구교통사고피해예측AI경진대회/submissions/submission_1.csv', index = False)